In [1]:
## INPUT
name="RiseOfTiamat"

In [2]:
players=dict(
{'Connor':'Parun',
 'Gruumsh':'Gruumsh',
 'Imelda':'DM',
 'Julie':'Ylva',
 'Lui W.':'Gruumsh',
 'Luise G.':'Gruumsh',
 'Mike G. (GM)':'DM',
 'Parun':'Parun',
 'Ylva Tritina':'Ylva',
 'Trinity':'Trinity',
'Lui, The Unwise Goat':'Gruumsh',
'Lui, They see me misty steppin\'':'Gruumsh',
'Lui':'Gruumsh',
 'Zim':'Zim',
 'Zam':'Zam',
 'Snek':'Snek'
}
)
plyrs=["Parun","Gruumsh","Ylva"]

In [ ]:
players=dict(
{'Connor':'Kallistē',
 'Lui':'Smallfolk',
 'Julie':'Bethrynne',
 'Kallistē Phlegethina':'Kallistē',
 'Kallistē':'Kallistē',
 'Smallfolk':'Smallfolk',
 'Bethrynne Mistwinter':'Bethrynne',
 'Bethrynne':'Bethrynne'
}
)
plyrs=["Bethrynne","Smallfolk","Kallistē"]

In [ ]:
players=dict(
{'Dasypus Bellow.':'Dasypus',
 'Edmund':'Nepelope',
 'Eljas O.':'Kääpiöinën',
 'Goat':'Wise Goat',
 'Hak-Bina':'Hak-Bina',
 'Korth':'Korth',
 'Kääpiöinën':'DM',
 'Lui':'Nura',
 'Mike W.':'Korth',
'Nepelope':'Nepelope',
'Nura':'Nura',
 'Roger G. (GM)':'DM',
 'Who is Rea':'Oelita',
 'Zoanne':'Hak-Bina'
}
)
plyrs=["Dasypus","Nepelope","Oelita","Nura","Hak-Bina","Korth","Kääpiöinën","Wise Goat"]

In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup

skills={'ACROBATICS',
 'ANIMAL HANDLING',
 'ARCANA',
 'ATHLETICS',
 'DECEPTION',
 'HISTORY',
 'INSIGHT',
 'INTIMIDATION',
 'INVESTIGATION',
 'MEDICINE',
 'NATURE',
 'PERCEPTION',
 'PERFORMANCE',
 'PERSUASION',
 'RELIGION',
 'SLEIGHT OF HAND',
 'STEALTH',
 'SURVIVAL'}

print("Loading chat log...")
soup = BeautifulSoup(open('raw/'+name+'.html', 'r').read(), 'html.parser')
msgs=soup.findAll('div',{'class' : re.compile('message \w+')})

print("Parsing messages...")
plyr=None
when=None
df = pd.DataFrame(columns=['Player','Timestamp','Skill','Result', 'Raw','Plus'])
for c,m in enumerate(msgs):
    if c % 1000 == 0:
        print(c+1,"/",len(msgs))
    
    who=m.findAll('span',{'class': 'by'})
    if who!=[]:
        plyr=who[0].renderContents()
        when=m.find('span',{'class': 'tstamp'}).renderContents()
    
    log=True
    skchk=m.findAll('div',{'class': ['sheet-rolltemplate-skill','sheet-rolltemplate-simple']})
    if skchk!=[]:   
        skrolls=skchk[0].find_all('div',{'class': ['sheet-roll','sheet-adv','sheet-solo']})
        if len(skrolls)==1:
            roll=skrolls[0]
        elif len(skrolls)==2:
            roll=skrolls[0] if skrolls[1].find('span',{'class': 'sheet-grey'})!=None else skrolls[1]
        else:
            log=False
        result=roll.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()
    
        tmpskill=skchk[0].find('div',{'class': 'sheet-label'}).renderContents()
        tmpskill=re.sub(r'[^ A-Z0123456789-]','',str(tmpskill)).split(' ')
        skill= [" ".join(tmpskill[:-1]),tmpskill[-1]] if len(tmpskill)>2 else tmpskill
        
        
        if (skill[0]!='') and log and (skill[0] in skills):
            df.at[c, 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
            df.at[c, 'Timestamp'] = when.decode('UTF-8')
            df.at[c, 'Skill'] = str(skill[0])
            df.at[c, 'Result'] = result.decode('UTF-8')
            df.at[c, 'Raw'] = str(int(result)-int(skill[1]))
            df.at[c, 'Plus'] = str(skill[1])

df["Timestamp"]=pd.to_datetime(df["Timestamp"])
df["Player"]=df["Player"].apply(lambda x: players[x])
df['Day'] = df['Timestamp'].dt.strftime('%d.%m.%Y')
df=df[df["Player"].isin(plyrs)]
df.to_csv("skills_"+name+".csv",index=False,sep=";")
df

Loading chat log...
Parsing messages...
1 / 5400
1001 / 5400
2001 / 5400
3001 / 5400
4001 / 5400
5001 / 5400


,Player,Timestamp,Skill,Result,Raw,Plus,Day
10,Gruumsh,2020-04-19 19:09:00,PERSUASION,24,18,6,19.04.2020
16,Gruumsh,2020-04-19 20:24:00,INVESTIGATION,10,11,-1,19.04.2020
17,Ylva,2020-04-19 20:34:00,PERCEPTION,25,19,6,19.04.2020
20,Gruumsh,2020-04-19 20:44:00,PERFORMANCE,17,14,3,19.04.2020
29,Gruumsh,2020-04-19 21:19:00,ATHLETICS,26,19,7,19.04.2020
...,...,...,...,...,...,...,...
5391,Ylva,2020-12-11 22:08:00,INSIGHT,12,9,3,11.12.2020
5394,Ylva,2020-12-11 22:22:00,HISTORY,21,16,5,11.12.2020
5395,Parun,2020-12-11 22:34:00,INSIGHT,19,10,9,11.12.2020
5396,Parun,2020-12-11 22:34:00,INSIGHT,16,7,9,11.12.2020
